## Introduction:

Main Goals:

1. Identify the recipients that will engage with the campaign.
2. Maximise the campaign’s revenue.


Comments

- The dataset contains only 5% of donors.
- The donations are usually smaller than $20.
- This data is quite noisy, high dimensional.
- There is an inverse relationship between the probability to donate and the amount donated.


Link for dataset and some analysis ==> 

https://github.com/rebordao/kdd98cup

https://github.com/bobbyantonio/KDD98/blob/master/CleanData.py

- Github solutions ==>
https://github.com/rebordao/kdd98cup


- Siraj notebook for a better data visualization:

https://www.youtube.com/watch?v=yQsOFWqpjkE

There is a notebook associated with this one contains the exploratory data analysis part .. 
http://localhost:8888/notebooks/Python%20codes/rewards_KDD_Exploratory_Analysis.ipynb

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.python.ops import rnn, rnn_cell
from sklearn.model_selection import train_test_split
import pandas as pd
import random

import csv
import sys

import matplotlib.pyplot as plt

%matplotlib inline

import random

## warnings
import warnings
warnings.filterwarnings("ignore")

In [5]:
def load_data():
    n = 10 #number of records in file
    s = 1 #desired sample size
    df = pd.read_csv('tuple.csv', header = None, skiprows = sorted(random.sample(range(n),n-s)))
    df.columns = ['r0','f0','m0','ir0','if0', 'a','r1', 'f1', 'm1','ir1','if1','rew']
    return df

In [6]:
## I have to run it over all the data not just 100 records
df = load_data()
df['a'] = np.random.uniform(0,11,(df['a'].shape)).astype(int)
df.head(3)

,r0,f0,m0,ir0,if0,a,r1,f1,m1,ir1,if1,rew
0,0,0,0.0,0,0,3,1,0,0.0,1,0,0.0
1,0,0,0.0,0,0,0,1,0,0.0,1,0,0.0
2,0,0,0.0,0,0,2,1,0,0.0,1,0,0.0


In [7]:
def df_split():
    
    ## I want to infer next_state from current_state, hence, learnt from LSTM .. I = s, O = s'
    df = load_data()
    train, test = train_test_split(df, test_size = 0.90)  # split data to 50-50 cross validate, Roger 1.6*1000,000
    
    train_x = train[train.columns.difference(['rew', 'a', 'r1', 'f1', 'm1', 'ir1', 'if1'])]
    train_y = train[train.columns.difference(['rew', 'a', 'r0', 'f0', 'm0', 'ir0', 'if0'])]
    
#     train_y = train_y.convert_objects(convert_numeric = True)
    
#     train_y = train_y.convert_objects(convert_numeric = True)
    
    return train_x, train_y, train

In [9]:
# df_split()

## Regression phase:

Before performing the prediction task .. let's split the data to training and validation sets .. 

To avoid any problems in prediction by having string variables .. let's binarize (catergorize) all the variables .. 

Guidance ==> https://pythonprogramming.net/rnn-tensorflow-python-machine-learning-tutorial/

In [40]:
gamma = 0.9
Num_itrs = 2  # no loop 3leha ( w dah el sa7) .. we just have to run the whole algo. 10 times and report the avg. results

num_epoch = 1 #23 #epochs are cycles of Feedforward and Backprob
## el mafrood yeb2a feh loop 3la el epochs elli heyya el steps .. w avg. reward per step is calculated 
batch_size = 2
chunkSize = 1

train_x, train_y, train = df_split()

n_nodes_hl1 = 40 #np.shape(train)[1]
n_nodes_hl2 = 15 #np.shape(train)[0]
# NUM_STATES = np.shape(train)[1]
NUM_DIM =  np.shape(train_x)[1]
num_nodes = np.shape(train_x)[0]
NUM_DIM_output = np.shape(train_y)[1]
########################################################################################################

# Exploration parameters for epsilon greedy strategy
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01            # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob
max_steps = 2
##########################################################################################################

num_unrollings = 5

# best_actions = np.zeros([np.shape(train)[0], batch_size])
Q_optimal = [] #np.zeros([np.shape(curr_state_current_action)[0], len(df['ACCOUNT_STATUS'].unique())])

########################################################################################################

## for training 
x = tf.placeholder(tf.float32, shape=[num_nodes, NUM_DIM])
y =  tf.placeholder(tf.float32, shape=[num_nodes, NUM_DIM_output])

# ## for testing
# x_ = tf.placeholder(tf.float32, shape=[NUM_DIM, None])
# y_ =  tf.placeholder(tf.float32, shape=[NUM_DIM, None])

########################################################################################################

# Input gate: input, previous output, and bias.
ix = tf.Variable(tf.random_uniform([NUM_DIM, NUM_DIM_output],0,5, dtype = tf.float32))# init_weights_RNN([n_nodes_hl1, NUM_ACTIONS])
im = tf.Variable(tf.random_uniform([NUM_DIM_output, NUM_DIM_output],0,5, dtype = tf.float32))
ib = tf.Variable(tf.zeros([1, NUM_DIM_output], dtype = tf.float32))

# Forget gate: input, previous output, and bias.
fx = tf.Variable(tf.random_uniform([NUM_DIM, NUM_DIM_output],0,5, dtype = tf.float32))
fm = tf.Variable(tf.random_uniform([NUM_DIM_output, NUM_DIM_output],0,5, dtype = tf.float32))
fb = tf.Variable(tf.zeros([1, NUM_DIM_output], dtype = tf.float32))

# Memory cell: input, state and bias.                             
cx = tf.Variable(tf.random_uniform([NUM_DIM, NUM_DIM_output],0,5, dtype = tf.float32))
cm = tf.Variable(tf.random_uniform([NUM_DIM_output, NUM_DIM_output],0,5, dtype = tf.float32))
cb = tf.Variable(tf.zeros([1, NUM_DIM_output], dtype = tf.float32))

# Output gate: input, previous output, and bias.
ox = tf.Variable(tf.random_uniform([NUM_DIM, NUM_DIM_output],0,5, dtype = tf.float32))
om = tf.Variable(tf.random_uniform([NUM_DIM_output, NUM_DIM_output],0,5, dtype = tf.float32))
ob = tf.Variable(tf.zeros([NUM_DIM_output], dtype = tf.float32))

# Variables saving state across unrollings.
saved_output = tf.Variable(tf.zeros([num_nodes, NUM_DIM_output], dtype = tf.float32), trainable = False) 

saved_state = tf.Variable(tf.zeros([num_nodes, NUM_DIM_output], dtype = tf.float32), trainable = False) 

# Classifier weights and biases.
w = tf.Variable(tf.random_uniform([NUM_DIM_output, NUM_DIM_output], 0, 5, dtype = tf.float32))
b = tf.Variable(tf.zeros([NUM_DIM_output], dtype = tf.float32))

In [41]:
# Definition of the cell computation.
# this method takes single cell and returns single number 
def lstm_cell(i, o, state):
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate*tf.tanh(state), state

In [42]:
def lstm_model(train_x, saved_output, saved_state):
    # Unrolled LSTM loop.    
#     output_ = list()
    output = saved_output  # row !
    state = saved_state  # row !

    
    output_, state = lstm_cell(train_x.values.astype(np.float32), tf.cast(output, tf.float32), tf.cast(state, tf.float32)) 
    
    # astype('U') .. to convert numpy array to string ..
#     for i in range(np.shape(train_x)[0]):## el loop faydetha to copy the next line that is just for single unit 
#         output_, state = lstm_cell(train_x.values[i, None].astype(np.float32), output, state) 

    
        # in case the last values are saved !
    with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
        model_output = tf.matmul(output_, w) + b # outputs single value
    
    
    return model_output ## the output for the whole model

In [43]:
def train_lstm_model():

    loss_RNN = []
    
    model_output = lstm_model(train_x, saved_output, saved_state) #output here is a vector

    cost = tf.reduce_mean(tf.square(y - model_output))

    optimize = tf.train.GradientDescentOptimizer(0.001).minimize(cost) 

    sess = tf.Session()
    init = tf.global_variables_initializer()  ## updated version from initialize_all_variables :) 
    sess.run(init)

    
    Predicted_states = sess.run(model_output, feed_dict={x:train_x.values.astype(np.float32), y:train_y.values.astype(np.float32)})
    
    # Cost calculation
    for step in range(1000):
        l,_ = sess.run([cost, optimize], feed_dict={x:train_x.values, y:train_y.values})
    
        if step % 100 == 0:
            loss_RNN.append(l)
     
    sess.close()            
    
#     print Predicted_states
    
    return Predicted_states  

## Learning the rewards:
Now, the LSTM_RNN output (from validation phase) is considered as input for DQN model, to be able to select the action that has the maximum longtime reward for the customer (highest CLV) .. 

https://arxiv.org/pdf/1602.01580.pdf

## Multilayer Deep Neural Network (DQN):

https://github.com/MorvanZhou/Reinforcement-learning-with-tensorflow/blob/master/contents/5_Deep_Q_Network/RL_brain.py

In [44]:
# I believe enn this funtion is to predict the actions .. w neb2a t7t wanna predict max Q 
def DQN_train(x_inputs):

    # Placeholder
    X = tf.placeholder(dtype=tf.float32, shape=[None, np.shape(x_inputs)[1]])
    Y = tf.placeholder(dtype=tf.float32, shape=[None])


    # Model architecture parameters
    n_dim = np.shape(x_inputs)[1] 
    n_neurons_1 = 40
    n_neurons_2 = 15
    n_target = 1 #np.shape(x_inputs)[0]
    
    epochs = 10
    
#     Qs_predicted = []
    
        # Initializers
    sigma = 1
    
    #First Q Network
    w1 = tf.Variable(tf.random_uniform([n_dim, n_neurons_1], 0, 0.1))
    bias1 = tf.Variable(tf.random_uniform([n_neurons_1], 0, 0.1))
    
    w2 = tf.Variable(tf.random_uniform([n_neurons_1, n_neurons_2], 0, 0.1))
    bias2 = tf.Variable(tf.random_uniform([n_neurons_2], 0, 0.1))
    
    w3 = tf.Variable(tf.random_uniform([n_neurons_2, n_target], 0, 0.1))
    bias3 = tf.Variable(tf.random_uniform([n_target], 0, 0.1))
    
    hidden_1 = tf.nn.relu(tf.matmul(X, w1) + bias1)
    hidden_2 = tf.nn.relu(tf.matmul(hidden_1, w2) + bias2)
    y_ = tf.matmul(hidden_2, w3) + bias3
    
    # initialize variables
    init_op = tf.global_variables_initializer()
    
    # Cost function
    mse = tf.reduce_mean(tf.squared_difference(y_, Y))

    # Optimizer
    opt = tf.train.RMSPropOptimizer(0.0001, 0.99).minimize(mse)

    with tf.Session() as sess:
        sess.run(init_op)
        num_itr = int(np.shape(x_inputs)[0]/batch_size)

        Qs_next_state = sess.run(y_, feed_dict={X: x_inputs})

    return Qs_next_state 

In [45]:
def predict_action(state, actions):
    
     # Placeholder
    X = tf.placeholder(dtype=tf.float32, shape=[None, np.shape(state)[1]])
    Y = tf.placeholder(dtype=tf.float32, shape=[None])


    # Model architecture parameters
    n_dim = np.shape(state)[1] 
    n_neurons_1 = 40
    n_neurons_2 = 15
    n_target = 11 #np.shape(x_inputs)[0]
    
    epochs = 2 #100
    
    Qs_next_state = []
    
    # Initializers
    sigma = 1
    
    #First Q Network
    w1 = tf.Variable(tf.random_uniform([n_dim, n_neurons_1], 0, 0.1))
    bias1 = tf.Variable(tf.random_uniform([n_neurons_1], 0, 0.1))
    
    w2 = tf.Variable(tf.random_uniform([n_neurons_1, n_neurons_2], 0, 0.1))
    bias2 = tf.Variable(tf.random_uniform([n_neurons_2], 0, 0.1))
    
    w3 = tf.Variable(tf.random_uniform([n_neurons_2, n_target], 0, 0.1))
    bias3 = tf.Variable(tf.random_uniform([n_target], 0, 0.1))
    
    
    hidden_1 = tf.nn.relu(tf.matmul(X, w1) + bias1)
    hidden_2 = tf.nn.relu(tf.matmul(hidden_1, w2) + bias2)
    y_ = tf.matmul(hidden_2, w3) + bias3
    # initialize variables
    init_op = tf.global_variables_initializer()
    
    # Cost function
    mse = tf.reduce_mean(tf.squared_difference(y_, Y))

    # Optimizer
    opt = tf.train.RMSPropOptimizer(0.0001, 0.99).minimize(mse)

    possible_actions = train['a'].unique() #list(range(12))

    ## EPSILON GREEDY STRATEGY
    # Choose action a from state s using epsilon greedy.
    ## First we randomize a number
#     exp_exp_tradeoff = np.random.rand() + 0.3

#     # Here we'll use an improved version of our epsilon greedy strategy used in Q-learning notebook
#     explore_probability = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * decay_step)
    # initialize variables
    init_op = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        sess.run(init_op)  
#         if (explore_probability >= exp_exp_tradeoff):
#             # Make a random action (exploration)
#             action = random.choice(possible_actions)

#         else:
            # Get action from Q-network (exploitation)
            # Estimate the Qs values state
        Qs = sess.run(y_, feed_dict = {X: state})
        # Take the biggest Q value (the best action)
        choice = max(np.argmax(Qs, axis = 1))
        action = possible_actions[int(choice)]
    return action

In [110]:
def Q_learning():
    
    avg_Q = []  
    gamma = 0.9
#     actions = actions_prep(df)

# nepisode > https://stats.stackexchange.com/questions/250943/what-is-the-difference-between-episode-and-epoch-in-deep-q-learning
# means one complete path from state, action, next_s, next_a, reward upuntil terminal state

    nepisod = 2 #2 #np.shape(actions)[1]  ##22+ 
    
    predicted_states = pd.DataFrame(train_lstm_model())

    
#     tuplesMx0, tuplesMx1, tuplesMx2, tuplesMx3, tuplesMx4, tuplesMx5, tuplesMx6, tuplesMx7, tuplesMx8, tuplesMx9, tuplesMx10, tuplesMx11 = tuple_(predicted_states) 
    
    num_rows = np.shape(train)[0] #-1
    
    Q_predicted = np.zeros([num_rows, 12]) #np.random.random
    
    best_action = np.zeros([num_rows, 1])
#     Q_optimal = np.zeros([nepisod, num_rows])
    decay_step = 0 
    
    
#     next_state = train[train.columns.difference(['rew', 'a', 'r0', 'f0', 'm0', 'ir0', 'if0'])]
    state = train[train.columns.difference(['rew', 'a', 'r1', 'f1', 'm1', 'ir1', 'if1'])]
    possible_actions = train['a'].unique()
    
    target_Qs_batch = []
    memory = []
    
    nsteps = 10
    
    #episod: is a complete game from start to end 
    for i in range(nepisod):
        episode_rewards = []
        for j in range(nsteps):
            action = predict_action(state, possible_actions)
            # find the reward of the corresponding action
            selected_a = train.loc[train['a'] == action, 'a']
            row = np.random.choice(selected_a, 1)  
            
            # Get the rewards corresponding to that action
            reward = np.array(train['rew'].iloc[row]) #game.make_action(action)  ## select the rewards corresponding to action a randomly 

            memory.append([state.iloc[row, :], action, predicted_states.iloc[row, :], reward])

            if (len(memory) <= batch_size):
                batch = pd.DataFrame(memory)
            else:
                batch = pd.DataFrame(random.sample(memory, batch_size)) # I can try np.random.permutation(memory)
                
            ## sample Random Mini-batch from D
#             so, this is the problem .. i wanna split batch to set of columns .. 
            states_mb = batch.iloc[:, 0:4] #np.array([each for each in batch], ndmin = 5)
            actions_mb = batch.loc[:, 4:5] #np.array([each[1] for each in batch])
            rewards_mb = batch.loc[:, 6:6] #np.array([each[2] for each in batch]) 
            next_states_mb = batch.loc[:, 7:11] #np.array([each[3] for each in batch], ndmin = 5)  
            dones_mb = random.randint(0, 1) #batch.loc[:, 12:12]  ##flag variable
            
#             next_states_mb = states_mb
            
            
            Qs_next_state = DQN_train(next_states_mb) 
            print(max(Qs_next_state))
#         for i in range(np.shape(batch)[0]):
#             terminal = dones_mb
            # If we are in a terminal state, only equals reward
#             if dones_mb:
#                 target_Qs_batch.append(rewards_mb.values[i])            
#             else:
#                 print(max(Qs_next_state))
#                 target = rewards_mb[i] + gamma * max(Qs_next_state[i])
#                 target_Qs_batch.append(target)                   
#                 targets_mb = np.array([each for each in Qs_next_state])
#                 best_action = np.argmax(targets_mb, axis = 1) 
                
#             print(target_Qs_batch)

    return best_action, _

In [ ]:
best_action, _ = Q_learning()
best_action

Exception ignored in: <bound method TF_Output.<lambda> of <tensorflow.python.pywrap_tensorflow_internal.TF_Output; proxy of <Swig Object of type 'TF_Output *' at 0x000002714F2D02D0> >>
Traceback (most recent call last):
  File "C:\Users\Eman\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 864, in <lambda>
    __del__ = lambda self: None
KeyboardInterrupt
Exception ignored in: <bound method TF_Output.<lambda> of <tensorflow.python.pywrap_tensorflow_internal.TF_Output; proxy of <Swig Object of type 'TF_Output *' at 0x000002714F2D02D0> >>
Traceback (most recent call last):
  File "C:\Users\Eman\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 864, in <lambda>
    __del__ = lambda self: None
KeyboardInterrupt
Exception ignored in: <bound method TF_Output.<lambda> of <tensorflow.python.pywrap_tensorflow_internal.TF_Output; proxy of <Swig Object of type 'TF_Output *' at 0x000002714F2D02D0> >>
Traceback (most recent call las